In [4]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
df = pd.read_csv('Avg_CRU.csv')
df.head()

,time,lat,lon,timeseries-tas-monthly-mean
0,1901-01-16,26.25,-122.75,NaN
1,1901-01-16,26.25,-122.25,NaN
2,1901-01-16,26.25,-121.75,NaN
3,1901-01-16,26.25,-121.25,NaN
4,1901-01-16,26.25,-120.75,NaN


In [38]:
df['time'] = pd.to_datetime(df['time'])

In [45]:
df.groupby(['lat', 'lon', pd.Grouper('time')])['timeseries-tas-monthly-mean'].resample('M').mean()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [6]:
df.shape

(6644352, 4)

In [7]:
df['time'] = pd.to_datetime(df['time'])

In [8]:
df.groupby(['lat', 'lon', 'time'])['timeseries-tas-monthly-mean'].mean()

lat    lon      time      
26.25  -122.75  1901-01-16          NaN
                1901-02-15          NaN
                1901-03-16          NaN
                1901-04-16          NaN
                1901-05-16          NaN
                                ...    
47.75  -71.25   2021-08-16    15.300000
                2021-09-16    10.200000
                2021-10-16     5.800000
                2021-11-16    -4.300000
                2021-12-16    -9.900001
Name: timeseries-tas-monthly-mean, Length: 6644352, dtype: float64

In [9]:
df['time'].min(), df['time'].max()

(Timestamp('1901-01-16 00:00:00'), Timestamp('2021-12-16 00:00:00'))

In [10]:
df_copy = df.copy()

In [11]:
df_copy['time'] = pd.to_datetime(df_copy['time'])
df_copy['year'] = df_copy['time'].dt.year
df_copy['month'] = df_copy['time'].dt.month
df_copy['dayofyear'] = df_copy['time'].dt.dayofyear
df_copy['quarter'] = df_copy['time'].dt.quarter

In [12]:
#test_file.drop(['timeseries-tas-monthly-mean', 'time'], axis=1, inplace=True)
#test_file.loc[:, 'timeseries-tas-monthly-mean'] = model.predict(test_file)
#test_file.head()

In [13]:
#final_file = pd.concat([test_file, train_file])
#final_file.drop('time', axis=1, inplace=True)

In [14]:
#final_file.head()

In [15]:
#final_file.isnull().sum()

In [17]:
#fig, ax = plt.subplots(figsize=(15, 6))
#df_1 = df_copy.loc[(df_copy['time'] >= '1901-01-16') & (df_copy['time'] < '1950-01-16'), :]
#sns.kdeplot(data=df_1, x='timeseries-tas-monthly-mean', label='Raw Data', fill=True, ax=ax)
#plt.legend()
#plt.show()

In [18]:
#fig, ax = plt.subplots(2, 1, figsize=(15, 6))
#df_1['timeseries-tas-monthly-mean'].plot(color='b', ax=ax[0])
#final_file['timeseries-tas-monthly-mean'].plot(color='r', ax=ax[1])
#plt.show()

In [19]:
#fig, ax = plt.subplots(figsize=(15, 6))
#df_1.groupby('year')['timeseries-tas-monthly-mean'].mean().plot(color='b', ax=ax)
#final_file.groupby('year')['timeseries-tas-monthly-mean'].mean().plot(color='r', ax=ax)
#plt.show()

In [20]:
#fig, ax = plt.subplots(figsize=(15, 6))
#df_1.groupby('year')['timeseries-tas-monthly-mean'].rolling(window=10).mean().plot(color='b', ax=ax)
#final_file.groupby('year')['timeseries-tas-monthly-mean'].rolling(window=10).mean().plot(color='r', ax=ax)
#plt.show()

In [21]:
def split_time_series(start_year, end_year, data=df_copy, col='timeseries-tas-monthly-mean'):
    data = data.loc[(data['time'] >= start_year) & ((data['time'] < end_year)), :]
    
    frame_to_use = data.loc[data[col].notna(), :]
    frame_to_pred = data.loc[data[col].isna(), :]
    
    X = frame_to_use.drop(['timeseries-tas-monthly-mean', 'time'], axis=1)
    y = frame_to_use['timeseries-tas-monthly-mean']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    prediction_val = model.predict(X_val)
    prediction_test = model.predict(X_test)
    
    frame_to_pred.drop(['timeseries-tas-monthly-mean', 'time'], axis=1, inplace=True)
    frame_to_pred.loc[:, 'timeseries-tas-monthly-mean'] = model.predict(frame_to_pred)
    
    return frame_to_pred

In [22]:
preds = split_time_series('1901-01-16', '1950-01-16')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_300\3563636579.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.drop(['timeseries-tas-monthly-mean', 'time'], axis=1, inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_300\3563636579.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.loc[:, 'timeseries-tas-monthly-mean'] = model.predict(frame_to_pred)


In [23]:
df_copy.loc[(df_copy['time'] >= '1901-01-16') & 
            (df_copy['time'] < '1950-01-16'), :] = df_copy.loc[(df_copy['time'] >= '1901-01-16') & 
            (df_copy['time'] < '1950-01-16'), :].fillna(preds, axis=0)

In [24]:
preds_1 = split_time_series('1950-01-16', '2001-01-16')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_300\3563636579.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.drop(['timeseries-tas-monthly-mean', 'time'], axis=1, inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_300\3563636579.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.loc[:, 'timeseries-tas-monthly-mean'] = model.predict(frame_to_pred)


In [25]:
df_copy.loc[(df_copy['time'] >= '1950-01-16') & 
            (df_copy['time'] < '2001-01-16'), :] = df_copy.loc[(df_copy['time'] >= '1950-01-16') & 
            (df_copy['time'] < '2001-01-16'), :].fillna(preds_1, axis=0)

In [26]:
preds_2 = split_time_series('2001-01-16', '2021-01-16')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_300\3563636579.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.drop(['timeseries-tas-monthly-mean', 'time'], axis=1, inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_300\3563636579.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.loc[:, 'timeseries-tas-monthly-mean'] = model.predict(frame_to_pred)


In [27]:
df_copy.loc[(df_copy['time'] >= '2001-01-16') & 
            (df_copy['time'] < '2021-01-16'), :] = df_copy.loc[(df_copy['time'] >= '2001-01-16') & 
            (df_copy['time'] < '2021-01-16'), :].fillna(preds_2, axis=0)

In [28]:
preds_3 = split_time_series('2021-01-16', '2021-12-16')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_300\3563636579.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.drop(['timeseries-tas-monthly-mean', 'time'], axis=1, inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_300\3563636579.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.loc[:, 'timeseries-tas-monthly-mean'] = model.predict(frame_to_pred)


In [29]:
df_copy.loc[(df_copy['time'] >= '2021-01-16') & 
            (df_copy['time'] <= '2021-12-16'), :] = df_copy.loc[(df_copy['time'] >= '2021-01-16') & 
            (df_copy['time'] <= '2021-12-16'), :].fillna(preds_3, axis=0)

In [30]:
print(df_copy.shape, df.shape)

(6644352, 8) (6644352, 4)


In [31]:
df.shape

(6644352, 4)

In [32]:
df_copy.shape

(6644352, 8)

In [33]:
df.isnull().sum()

time                                 0
lat                                  0
lon                                  0
timeseries-tas-monthly-mean    1209516
dtype: int64

In [34]:
df_copy.isnull().sum()

time                             0
lat                              0
lon                              0
timeseries-tas-monthly-mean    833
year                             0
month                            0
dayofyear                        0
quarter                          0
dtype: int64

In [35]:
df_copy.loc[df_copy['timeseries-tas-monthly-mean'].isnull(), :]

,time,lat,lon,timeseries-tas-monthly-mean,year,month,dayofyear,quarter
6639776,2021-12-16,26.25,-122.75,NaN,2021,12,350,4
6639777,2021-12-16,26.25,-122.25,NaN,2021,12,350,4
6639778,2021-12-16,26.25,-121.75,NaN,2021,12,350,4
6639779,2021-12-16,26.25,-121.25,NaN,2021,12,350,4
6639780,2021-12-16,26.25,-120.75,NaN,2021,12,350,4
...,...,...,...,...,...,...,...,...
6642789,2021-12-16,40.25,-72.25,NaN,2021,12,350,4
6642790,2021-12-16,40.25,-71.75,NaN,2021,12,350,4
6642791,2021-12-16,40.25,-71.25,NaN,2021,12,350,4
6642894,2021-12-16,40.75,-71.75,NaN,2021,12,350,4


In [36]:
df_copy.to_csv('Updated_CRU.csv', index=False)

In [238]:
import numpy as np
from scipy.optimize import minimize

df_1 = df.loc[(df['time'] >= '1901-01-16') & (df['time'] < '1910-01-16'), :]

df_1[df_1 == -9999] = np.nan

def neg_log_likelihood(params, data):
    mu, sigma = params
    n = len(data)
    data = data[~np.isnan(data)]
    return n*np.log(sigma) + np.nansum((data-mu)**2)/(2*sigma**2)

mu_init = np.nanmean(df_1)
sigma_init = np.nanstd(df_1)
params_init = [mu_init, sigma_init]

# Maximize the likelihood function
result = minimize(neg_log_likelihood, params_init, args=(df_1,), method='BFGS')

# Extract the parameter estimates
mu_est, sigma_est = result.x

C:\Users\lenovo\AppData\Local\Temp\ipykernel_8684\2758902732.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[df_1 == -9999] = np.nan
C:\Users\lenovo\AppData\Local\Temp\ipykernel_8684\2758902732.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[df_1 == -9999] = np.nan


TypeError: can only concatenate str (not "float") to str

In [ ]:
import pandas as pd
import numpy as np

# create a DataFrame with some missing temperature data
df = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04'],
                   'temperature': [32, np.nan, 28, np.nan]})

# define a rolling window size
window = 2

# fill in the missing temperature data with the rolling mean
df['temperature'].fillna(df['temperature'].rolling(window).mean(), inplace=True)

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Create a DataFrame with some missing temperature data
df = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04'],
                   'temperature': [32, np.nan, 28, np.nan]})

# Fit an ExponentialSmoothing model
model = ExponentialSmoothing(df['temperature'], trend='add', seasonal='add', seasonal_periods=12).fit()

# make predictions
predictions = model.predict(start=len(df['temperature'])-1, end=len(df['temperature']))

# fill missing temperature data with the predicted values
df['temperature'].fillna(predictions, inplace=True)

In [ ]:
!pip install fbprophet

from fbprophet import Prophet
import pandas as pd

# Create a DataFrame with historical temperature data
df = pd.DataFrame({'ds': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04'],
                   'y': [32, np.nan, 28, np.nan]})

# Initialize the Prophet model
m = Prophet()

# Fit the model to the historical data
m.fit(df)

# Create a DataFrame for future dates
future = m.make_future_dataframe(periods=1, freq='D')

# Make predictions for the future dates
forecast = m.predict(future)

# Extract the predicted values for the temperature
predicted_temperature = forecast[['ds', 'yhat']].tail(1)

# Fill in the missing temperature data with the predicted values
df['y'].fillna(predicted_temperature['yhat'].values[0], inplace=True)

In [ ]:
import pystan
import pandas as pd

# Load the data into a pandas DataFrame
df = pd.read_csv("data.csv")

# Define the Stan model
model_code = """
data {
    int<lower=0> N;
    int<lower=0> K;
    int<lower=1,upper=K> x[N];
    vector[K] mu;
    matrix[K,K] Sigma;
}
parameters {
    vector[K] beta;
}
model {
    for (n in 1:N)
        x[n] ~ categorical(mu[x[n]]);
    beta ~ multi_normal(mu, Sigma);
}
"""

# Compile the model
model = pystan.StanModel(model_code=model_code)

# Prepare the data for the model
data = {'N': df.shape[0],
        'K': df.shape[1],
        'x': df.values+1,  # missing values are replaced with -1
        'mu': df.mean(),
        'Sigma': df.cov()}

# Fit the model
fit = model.sampling(data=data, iter=1000, chains=4)

# Extract the parameter estimates
beta_estimates = fit.extract()['beta']

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Load the data into a numpy array
data = np.loadtxt("data.txt")

# Replace missing values with NaNs
data[data == -9999] = np.nan

# Define a function to calculate the negative log-likelihood
def neg_log_likelihood(params, data):
    mu, sigma = params
    n = len(data)
    data = data[~np.isnan(data)]
    return n*np.log(sigma) + np.nansum((data-mu)**2)/(2*sigma**2)

# Initialize the parameter estimates
mu_init = np.nanmean(data)
sigma_init = np.nanstd(data)
params_init = [mu_init, sigma_init]

# Maximize the likelihood function
result = minimize(neg_log_likelihood, params_init, args=(data,), method='BFGS')

# Extract the parameter estimates
mu_est, sigma_est = result.x